# Sección 2. Entendimiento y Preparación de los datos

Esta sección tiene como propósito exponer el entendimiento y la preparación de los textos para posteriormente utilizarlos en la implementación de los modelos de aprendizaje. Para la manipulación de los datos se usarán librerias como **pandas** y **numpy**. Para la visualización se utilizarán **matplotlib**, **seaborn** y **WordCloud**.

## 1. Instalación e importación de librerias

### Instalación

In [ ]:
#%pip install pandas numpy matplotlib seaborn scikit-learn nltk spacy wordcloud tqdm
#%pip install spacy-langdetect
#!python -m spacy download es_core_news_sm
#%pip install tensorflow
#%pip install keras
#!pip install wordcloud
#!pip install spacy-langdetect
#!python -m spacy download es_core_news_sm
#!pip install tensorflow
#!pip install keras
#!pip install spacy


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\mgs05\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\mgs05\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\mgs05\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe: No module named spacy



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\mgs05\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\mgs05\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached cloudpathlib-0.20.0-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ------------------ --------------------- 5.8/12.2 MB 32.0 MB/s eta 0:00:01
   ------------------------------------


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\mgs05\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Procesamiento de texto
import re
import string
import spacy
from spacy_langdetect import LanguageDetector
import re
import unicodedata

import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score, ConfusionMatrixDisplay,precision_recall_curve
from sklearn.tree import DecisionTreeClassifier

# Progreso en bucles
from tqdm import tqdm

from collections import Counter
#from ydata_profiling import ProfileReport
from scipy.sparse import hstack

import numpy as np

# Tensor flow para redes neuronales
import tensorflow as tf
from tensorflow import keras


# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Descargar stopwords de NLTK en español
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10,5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## 2. Perfilamiento y entendimiento de los datos

### Lectura de los datos

Procedemos a cargar los datos y a revisar su estructura.

In [ ]:
# Cargar datos
data=pd.read_csv('fake_news_spanish.csv', sep=';', encoding = "utf-8")

news_df = data.copy()

In [ ]:
news_df.info()  

In [ ]:
news_df.head()

Entendemos entonces, que tenemos una variable objetivo llamada *'Label'* la cual determina si una historia es falsa o no. Adicionalmente, identificamos otras variables como el *'Titulo'* de la historia y su *'Descripción'*. Adicionalmente, la *'Fecha'* es probablemente la fecha de publicación. 

### Entendimiento de los Datos

#### Estadísticas descriptivas

El entendimiento de los datos consiste en analizar las estadísticas que describen nuestras variables. En este sentido, haremos un conteo preliminar de las palabras en el título de cada noticia, la moda, el mínimo y el máximo.

In [ ]:
news_df['Conteo'] = [len(str(x)) for x in news_df['Titulo']]
news_df['Moda'] = [Counter([word.lower().strip(string.punctuation) for word in (str(i).split(' '))]).most_common()[0][0] for i in news_df['Titulo']]
news_df['Max'] = [[max([len(x) for x in str(i).split(' ')])][0] for i in news_df['Titulo']]
news_df['Min'] = [[min([len(x) for x in str(i).split(' ')])][0] for i in news_df['Titulo']]

news_df.head()

#### Perfilamiento de los datos

En cuanto al perfilamiento de los datos, se hará uso de ydata profiling para ir aún más a fondo en los datos que buscamos describir.

In [ ]:
##profile = ProfileReport(news_df, title="Pandas Profiling Report")
##profile

Como podemos observar, debemos ser muy cuidadosos a la hora de analisar el tema de datos duplicados y demás estadísticas cuando manejamos datos de texto. Hacer una limpieza y preparar los datos vale la pena teniendo en cuenta que la mayoria de valores obtenidos se ven alterados por lo que denominaremos como ruido.

## 3. Preparación de los datos

#### 3.1. Limpieza de datos.

Este código implementa un proceso de preprocesamiento de texto para limpiar datos textuales antes de realizar análisis de texto o procesamiento de lenguaje natural.El preprocesamiento se aplica a dos columnas de un DataFrame (Titulo y Descripcion), generando nuevas columnas con los textos limpios (Titulo_Limpio y Descripcion_Limpia). Este paso es fundamental porque ayuda a reducir el ruido en los datos, mejora la precisión de los modelos y facilita la extracción de información relevante eliminando términos irrelevantes o redundantes.


In [ ]:
import nltk
import unicodedata
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

news_df = news_df[["Label", "Titulo", "Descripcion"]]

# Funciones de preprocesamiento
def remove_non_ascii(words):
    """Remueve caracteres no ASCII de una lista de palabras"""
    return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words if word]

def to_lowercase(words):
    """Convierte todas las palabras a minúsculas"""
    return [word.lower() for word in words if word]

def remove_punctuation(words):
    """Elimina signos de puntuación"""
    return [re.sub(r'[^\w\s]', '', word) for word in words if re.sub(r'[^\w\s]', '', word) != '']

def remove_numbers(words):
    """Elimina los números de la lista de palabras"""
    return [word for word in words if not word.isdigit()]

def remove_stopwords(words):
    """Elimina stopwords en español"""
    return [word for word in words if word not in stopwords_es]

def preprocessing(text):
    """Aplica todas las funciones de limpieza de texto a un string"""
    words = word_tokenize(text, language="spanish")  # Tokenizar con NLTK
    words = to_lowercase(words)
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return " ".join(words)  # Retorna el texto limpio como un string

# Aplicar limpieza en el DataFrame
news_df['Titulo_Limpio'] = news_df['Titulo'].astype(str).apply(preprocessing)
news_df['Descripcion_Limpia'] = news_df['Descripcion'].astype(str).apply(preprocessing)

# Comparación antes y después
news_df.head()


#### 3.2. Tokenización

Este código realiza la tokenización de texto. Esta tokenización se aplica a las columnas ya limpias del DataFrame (Titulo_Limpio y Descripcion_Limpia), generando nuevas columnas Titulo_Tokens y Descripcion_Tokens, donde cada texto se representa como una lista de palabras. Este paso es crucial porque permite a los modelos de análisis de texto procesar los datos de manera estructurada.


In [ ]:
def tokenize_nltk(text):
    return word_tokenize(text, language="spanish")

# Aplicar tokenización en las columnas ya limpias
news_df['Titulo_Tokens'] = news_df['Titulo_Limpio'].astype(str).apply(tokenize_nltk)
news_df['Descripcion_Tokens'] = news_df['Descripcion_Limpia'].astype(str).apply(tokenize_nltk)

# Mostrar ejemplos
print("Comparación después de la tokenización:")
news_df.head()



#### 3.3. Lematización

Este código realiza la normalización del texto aplicando técnicas de stemming y lemmatización. La normalización se aplica a las columnas del DataFrame (Titulo_Tokens y Descripcion_Tokens), generando nuevas columnas Titulo_Normalizado y Descripcion_Normalizada, donde cada palabra es reducida a su raíz. Este paso es fundamental porque mejora la consistencia del texto y facilita el procesamiento de datos en modelos de análisis de texto.

In [ ]:


# Descargar recursos necesarios para lematización
#nltk.download('wordnet')
#nltk.download('omw-1.4')

# Crear los objetos para stemming y lematización
stemmer = SnowballStemmer("spanish")
lemmatizer = WordNetLemmatizer()

def stem_words(words):
    """Aplica stemming para eliminar prefijos y sufijos en las palabras"""
    return [stemmer.stem(word) for word in words]

def lemmatize_verbs(words):
    """Aplica lematización para obtener la raíz de los verbos"""
    return [lemmatizer.lemmatize(word, wordnet.VERB) for word in words]

def stem_and_lemmatize(words):
    """Combina stemming y lematización para normalizar los datos"""
    stemmed = stem_words(words)
    lemmatized = lemmatize_verbs(words)
    return list(set(stemmed + lemmatized))  # Se usa set() para evitar duplicados

# Aplicar normalización a las columnas tokenizadas
news_df['Titulo_Normalizado'] = news_df['Titulo_Tokens'].apply(stem_and_lemmatize)
news_df['Descripcion_Normalizada'] = news_df['Descripcion_Tokens'].apply(stem_and_lemmatize)

# Mostrar ejemplos
print("Comparación después de la normalización:")
news_df.head()


Se mostrarán solamente: Los tokens del título, los tokens de la descripción, el titulo normalizado y la descripción normalizada.

In [ ]:
processed_df = news_df[['Label', 'Titulo_Tokens', 'Descripcion_Tokens', 'Titulo_Normalizado', 'Descripcion_Normalizada']]
processed_df.head()

Ahora unificaremos los titulos y las descripciones normalizadas.

In [ ]:
# Hacer copia del dataframe procesado
processed_news_df = processed_df.copy()

# Convertir las listas de palabras en texto separado por espacios en las columnas Titulo_Normalizado y Descripcion_Normalizada
processed_news_df['Titulo_Normalizado'] = processed_news_df['Titulo_Normalizado'].apply(lambda tokens: " ".join(tokens))
processed_news_df['Descripcion_Normalizada'] = processed_news_df['Descripcion_Normalizada'].apply(lambda tokens: " ".join(tokens))

# Mostrar algunos ejemplos para verificar el cambio
processed_news_df.head()

## 4. Construcción de Modelos

Como los datos que tenemos son datos de texto, debemos de alguna maner convertir estos datos en valores interpretables por el modelo para generar información relevante que podamos interpretar. En este caso, usaremos a nuestro favor la **vectorización de texto** a partir del método TF-IDF ya que es una opción balanceada entre simplicidad y efectividad. La idea es convertir los textos de 'Titulo_Normalizado' y 'Descripcion_Normalizada' en vectores numéricos usando TF-IDF y luego usarlos para entrenar un modelo de clasificación. 

#### 4.1 Transformación TF-IDF

In [ ]:
# Definimos el vectorizador TF-IDF
vectorizer = TfidfVectorizer()  # Limitamos el número de características para evitar alta dimensionalidad

# Aplicamos la transformación TF-IDF a las columnas normalizadas
titulo_tfidf = vectorizer.fit_transform(processed_news_df['Titulo_Normalizado'])
descripcion_tfidf = vectorizer.fit_transform(processed_news_df['Descripcion_Normalizada'])

# Concatenamos ambas representaciones para tener una única matriz de características
X = hstack([titulo_tfidf, descripcion_tfidf])
# Definimos la variable objetivo
y = processed_news_df['Label']

# Mostramos la forma de la matriz final
print("Dimensión de la matriz de características X:", X.shape)
print("Dimensión de la variable objetivo y:", y.shape)

In [ ]:
# AUXILIAR

# Aplicamos la transformación TF-IDF a las columnas normalizadas
titulo_tfidf = vectorizer.fit_transform(processed_news_df['Titulo_Normalizado'])
descripcion_tfidf = vectorizer.fit_transform(processed_news_df['Descripcion_Normalizada'])

# Concatenamos ambas representaciones para tener una única matriz de características
X = hstack([titulo_tfidf, descripcion_tfidf])

#### 4.2 Entrenamiento del primer modelo de clasificación - Regresión Logística

Utilizaremos un modelo clásico de Regresión Logística, ya que es simple, eficiente y suele funcionar bien en tareas de clasificación de texto.

In [ ]:
# Dividimos los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Inicializamos y entrenamos el modelo de Regresión Logística
modelo = LogisticRegression(max_iter=1000)  # Aumentamos las iteraciones para asegurar convergencia
modelo.fit(X_train, y_train)

# Hacemos predicciones sobre el conjunto de prueba
y_pred = modelo.predict(X_test)

# Evaluamos el modelo con una matriz de confusión y métricas de desempeño
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

#### 4.3 Resultados de la Regresión Logística

##### 4.3.1 Reporte de Clasificación

In [ ]:
# Mostramos los resultados
print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test,y_pred)))
print("Precisión: {}".format(precision_score(y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred)))

print("\nReporte de Clasificación:")
print(classification_rep)

**Análisis de Métricas**
- **Exactitud:** El 89% de las noticias fueron clasificadas correctamente. Sin embargo, esta métrica no siempre refleja bien el rendimiento en problemas desbalanceados.

- **Recall:** El 96% de las noticias verdaderas fueron correctamente identificadas como verdaderas. El 79% de las noticias falsas fueron correctamente identificadas como falsas. El problema principal es que el modelo no está detectando bien las noticias falsas (21% de error en recall para 0.)

- **Presición:** El 86% de las noticias clasificadas como verdaderas realmente son verdaderas (lo cual indica que hay un error del 14%). el 94% de las noticias clasificadas como falsas realmente son falsas.

- **Puntuación F1:** Un valor más alto significa un mejor balance entre detectar correctamente noticias falsas y evitar clasificar noticias reales como falsas. El modelo tiene un mejor desempeño en la detección de noticias verdaderas (91%) que en la detección de noticias falsas (86%). Esto significa que el modelo puede pasar muchas noticias falsas como verdaderas (falsos negativos).

##### 4.3.2 Matríz de Confusión

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=modelo.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()

**Explicación de los valores del modelo**

- **Verdaderos Negativos (3773):** Noticias falsas *correctamente* identificadas como falsas.

- **Falsos Positivos (1008):** Noticias verdaderas clasificadad *incorrectamente* como falsas.

- **Falsos Negativos (244):** Noticias falsas clasificadas *incorrectamente* como verdaderas.

- **Verdaderos Positivos (6388):** Noticias verdaderas *correctamente* identificadas como verdaderas.

##### 4.3.3 Conclusiones iniciales

1. El modelo es confiable para detectar noticias verdaderas (96% recall), pero tiene problemas a la hora de detectar noticias falsas (solo 79% de recall).

2. El principal problema son los falsos positivos (1,008 noticias verdaderas clasificadas erróneamente como falsas).

3. Si el objetivo es detectar la mayor cantidad posible de noticias falsas, el modelo debe ser ajustado para mejorar su recall.

##### 4.3.4 Modificación del Umbral de clasificación

Actualmente, el umbral de clasificación para la detección de noticias es de 0.5, esto significa que si la probabilidad de una noticia de ser verdadera es mayor al 50% se clasifica como verdadera. Podemos entonces, disminuir este umbral para que el modelo sea más estricto al clasificar noticias como verdaderas, aumentando así la detección de noticias falsas.

In [ ]:
# Obtenemos las probabilidades de predicción en el conjunto de prueba
y_prob = modelo.predict_proba(X_test)[:, 1]  # Probabilidad de ser noticia verdadera (Clase 1)

# Calculamos Precision, Recall y Umbrales automáticamente
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

# Graficamos Precision-Recall para diferentes umbrales
plt.figure(figsize=(8, 6))
plt.plot(thresholds, precisions[:-1], label="Precisión")
plt.plot(thresholds, recalls[:-1], label="Recall")
plt.xlabel("Umbral de clasificación")
plt.ylabel("Métrica")
plt.legend()
plt.title("Precisión y Recall en función del Umbral de Clasificación")
plt.grid()
plt.show()


**Análisis de la curva Presición-Recall en función del Umbral de Clasificación**

El punto de equilibrio lo encontramos entre 0.6 y 0.7 podemos empezar a experimentar con estos valores y evaluar los resultados.

In [ ]:
# Definimos el nuevo umbral
nuevo_umbral = 0.6

# Aplicamos el nuevo umbral a las predicciones
y_pred_ajustado = (y_prob >= nuevo_umbral).astype(int)

# Calculamos las nuevas métricas
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

conf_matrix_ajustada = confusion_matrix(y_test, y_pred_ajustado)
accuracy_ajustada = accuracy_score(y_test, y_pred_ajustado)
classification_rep_ajustado = classification_report(y_test, y_pred_ajustado)

# Mostramos los resultados ajustados
print("Matriz de Confusión (Umbral 0.6):")
print(conf_matrix_ajustada)
print("\nReporte de Clasificación (Umbral 0.6):")
print(classification_rep_ajustado)
print(f"\nExactitud del modelo (Umbral 0.6): {accuracy_ajustada:.4f}")


Lo primero que identificamos es que el 93% de las noticias verdaderas fueron correctamente identificadas como verdaderas. El **84% de las noticias falsas fueron correctamente identificadas como falsas**, reduciendo así el error a 16%. 

In [ ]:
cm_modificada = confusion_matrix(y_test, y_pred_ajustado)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_modificada, display_labels=modelo.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()

La matríz de confusión también presenta resultados bastante satisfactorios, reduciendo los falsos negativos a cambio de el aumento en los falsos positivos, pero a su vez, aumentando también las noticias tanto falsas como verdaderas que se clasificaron correctamente. Concluimos un buen tradeoff y un modelo que puede cumplir con los estándares del negocio.

#### 4.4. Red neuronal

In [ ]:
# Dividimos los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definimos la arquitectura de la red neuronal
modelo_nn = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    keras.layers.Dropout(0.3),  # Regularización para evitar sobreajuste
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')  # Capa de salida para clasificación binaria
])

# Compilamos el modelo
modelo_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamos el modelo
history = modelo_nn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluamos el modelo en el conjunto de prueba
y_pred_prob = modelo_nn.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Calculamos métricas de evaluación
conf_matrix_nn = confusion_matrix(y_test, y_pred)
accuracy_nn = accuracy_score(y_test, y_pred)
classification_rep_nn = classification_report(y_test, y_pred)

# Mostramos los resultados
print("Matriz de Confusión:")
print(conf_matrix_nn)
print("\nReporte de Clasificación:")
print(classification_rep_nn)
print(f"\nExactitud del modelo: {accuracy_nn:.4f}")